In [111]:
class RecurrentNeuralNetwork:
    
    def __init__ (self, input, output, recurrences, expected_output, learning_rate):
        #initial input 
        self.x = np.zeros(input)
        #print('INPUT',self.x)
        
        #input size 
        self.input = input
        
        #expected output 
        self.y = np.zeros(output)
        #print('OUTPUT',self.y)
        
        #output size
        self.output = output
        
        
        #weight matrix 
        self.w = np.random.random((input, output))
        #print('WEIGHT',self.w)
        
        #matrix used in RMSprop in order to decay the learning rate
        self.G = np.zeros_like(self.w)
        #print('G->matrix used in RMSprop in order to decay the learning rate',self.G)
        
        #length of the recurrent network
        self.recurrences = recurrences
        
        #learning rate 
        self.learning_rate = learning_rate
        
        #array for storing inputs
        self.ia = np.zeros((recurrences+1,input))
        #print('ia->array for storing inputs',self.ia)
        
        #array for storing cell states
        self.ca = np.zeros((recurrences+1,output))
        #print('Ca------>array for storing cell states',self.ca)
        
        #array for storing outputs
        self.oa = np.zeros((recurrences+1,output))
        #print("oa--->array for storing outputs",self.oa)
        
        #array for storing hidden states
        self.ha = np.zeros((recurrences+1,output))
        
        #print("ha-->array for storing hidden state",self.ha) 
        
        #forget gate 
        self.af = np.zeros((recurrences+1,output))
        #print('af-------> Forget gate',self.af)

        
        #input gate
        self.ai = np.zeros((recurrences+1,output))
        #print('ai----->input gate',self.ai)
        
        #cell state
        self.ac = np.zeros((recurrences+1,output))
        #print('ac----->cell state',self.ai)
        #output gate
        self.ao = np.zeros((recurrences+1,output))
        #print('ao----->output gate',self.ao)
        
        #array of expected output values
        self.expected_output = np.vstack((np.zeros(expected_output.shape[0]), expected_output.T))
        #print('self.expected_output----->expected output value ',self.expected_output)
        #declare LSTM cell 
        print('--------------------------------------Initilizing LSTM---------------------')
        self.LSTM = LSTM(input, output, recurrences, learning_rate)
        
###################################################ADDED METHODs###############################################################
    def setInput(self,input_data):
        self.x=input_data
    def setTarget(self,target):
        self.y=target
        
    def summary(self):
        print('INPUT',self.x) 
        print('OUTPUT',self.y)   
        print('WEIGHT',self.w)
        print('G->matrix used in RMSprop in order to decay the learning rate',self.G)
        print('ia->array for storing inputs',self.ia)
        print('Ca------>array for storing cell states',self.ca)
        print("oa--->array for storing outputs",self.oa)
        print("ha-->array for storing hidden state",self.ha) 
        print('af-------> Forget gate',self.af)
        print('ai----->input gate',self.ai)
        print('ac----->cell state',self.ai)
        print('ao----->output gate',self.ao)
        print('self.expected_output----->expected output value ',self.expected_output)
      
        
    
        pass
    
##############################################################################################################################
    
    #sigmoid activation function
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    #derivative of sigmoid 
    def dsigmoid(self, x):
        return self.sigmoid(x) * (1 - self.sigmoid(x))    
    
    #Forward Propagation
    def forwardProp(self):
        for i in range(1, self.recurrences+1):
            self.LSTM.x = np.hstack((self.ha[i-1], self.x))
            cs, inp,hs, f, c, o = self.LSTM.forwardProp()
            #store cell state from the forward propagation
            self.ca[i] = cs #cell state
            self.ha[i] = hs #hidden state
            self.af[i] = f #forget state
            self.ai[i] = inp #inpute gate
            self.ac[i] = c #cell state
            self.ao[i] = o #output gate
            self.oa[i] = self.sigmoid(np.dot(self.w, hs)) #activate the weight*input
            self.x = self.expected_output[i-1]
        return self.oa
   
    # Back propagation
    def backProp(self):
        totalError = 0
        #cell state
        dfcs = np.zeros(self.output)
        #hidden state,
        dfhs = np.zeros(self.output)
        #weight matrix
        tu = np.zeros((self.output,self.output))
        #forget gate
        tfu = np.zeros((self.output, self.input+self.output))
        #input gate
        tiu = np.zeros((self.output, self.input+self.output))
        #cell unit
        tcu = np.zeros((self.output, self.input+self.output))
        #output gate
        tou = np.zeros((self.output, self.input+self.output))
        for i in range(self.recurrences, -1, -1):
            error = self.oa[i] - self.expected_output[i]
            tu += np.dot(np.atleast_2d(error * self.dsigmoid(self.oa[i])), np.atleast_2d(self.ha[i]).T)
            error = np.dot(error, self.w)
            self.LSTM.x = np.hstack((self.ha[i-1], self.ia[i]))
            self.LSTM.cs = self.ca[i]
            fu, iu, cu, ou, dfcs, dfhs = self.LSTM.backProp(error, self.ca[i-1], self.af[i], self.ai[i], self.ac[i], self.ao[i], dfcs, dfhs)
            totalError += np.sum(error)
            #forget gate
            tfu += fu
            #input gate
            tiu += iu
            #cell state
            tcu += cu
            #output gate
            tou += ou   
        self.LSTM.update(tfu/self.recurrences, tiu/self.recurrences, tcu/self.recurrences, tou/self.recurrences)  
        self.update(tu/self.recurrences)
        return totalError
    
    def update(self, u):
        self.G = 0.95 * self.G + 0.1 * u**2  
        self.w -= self.learning_rate/np.sqrt(self.G + 1e-8) * u
        return
    
    def sample(self):
        for i in range(1, self.recurrences+1):
            self.LSTM.x = np.hstack((self.ha[i-1], self.x))
            cs, hs, f, inp, c, o = self.LSTM.forwardProp()
            maxI = np.argmax(self.x)
            self.x = np.zeros_like(self.x)
            self.x[maxI] = 1
            self.ia[i] = self.x 
            #store cell states
            self.ca[i] = cs
            #store hidden state
            self.ha[i] = hs
            #forget gate
            self.af[i] = f
            #input gate
            self.ai[i] = inp
            #cell state
            self.ac[i] = c
            #output gate
            self.ao[i] = o
            self.oa[i] = self.sigmoid(np.dot(self.w, hs))
            maxI = np.argmax(self.oa[i])
            newX = np.zeros_like(self.x)
            newX[maxI] = 1
            self.x = newX
           
        return self.oa

In [112]:
class LSTM:
    # LSTM cell (input, output, amount of recurrence, learning rate)
    def __init__ (self, input, output, recurrences, learning_rate):
        #input size
        self.x = np.zeros(input+output)
        #print('INPUT',self.x)
        
        #input size
        self.input = input + output
        #output 
        self.y = np.zeros(output)
        #print('OUTPUT',self.y)
        #output size
        self.output = output
        #cell state intialized as size of prediction
        self.cs = np.zeros(output)
        #print('InitilizFing cell state',self.cs)
        
        #how often to perform recurrence
        self.recurrences = recurrences
        #balance the rate of training (learning rate)
        self.learning_rate = learning_rate
        #init weight matrices for our gates
        #forget gate
        self.f = np.random.random((output, input+output))
        #print('f-------> Forget gate',self.f)
        #input gate
        self.i = np.random.random((output, input+output))
        #print('f-------> input gate',self.i)
        #cell state
        self.c = np.random.random((output, input+output))
        #print('c-------> cell state',self.c)
        
        #output gate
        self.o = np.random.random((output, input+output))
        #print('o-------> output gate',self.o)
        #forget gate gradient
        self.Gf = np.zeros_like(self.f)
        #print('gf-------> gate gradient',self.Gf)
        
        #input gate gradient
        self.Gi = np.zeros_like(self.i)
        #print('Gi-------> input gate gradient',self.Gi)
        
        #cell state gradient
        self.Gc = np.zeros_like(self.c)
        #print('Gc----->Cell state gradient',self.Gc)
        
        #output gate gradient
        self.Go = np.zeros_like(self.o)
        #print('Go----->Outputgate gradient',self.Go)
      
        ###################################################ADDED METHODs###############################################################
    def setInput(self,input_data):
        self.x=input_data
    def setTarget(self,target):
        self.y=target
        
    def summary(self):       
        print('INPUT',self.x)       
        
        print('OUTPUT',self.y)       
        print('Initilizing cell state',self.cs)       
        
        print('f-------> Forget gate',self.f)       
        
        print('i-------> input gate',self.i)      
        
        print('c-------> cell state',self.c)        
       
        print('o-------> output gate',self.o)        
        print('gf-------> gate gradient',self.Gf)        
        
        print('Gi-------> input gate gradient',self.Gi)        
       
        print('Gc----->Cell state gradient',self.Gc)       
        
        print('Go----->Outputgate gradient',self.Go)   
        
    
        pass
    
##############################################################################################################################
    
   
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def dsigmoid(self, x):
        return self.sigmoid(x) * (1 - self.sigmoid(x))
    
    
    def tangent(self, x):
        return np.tanh(x)
    
    
    def dtangent(self, x):
        return 1 - np.tanh(x)**2
  
    def forwardProp(self):
        f = self.sigmoid(np.dot(self.f, self.x))
        self.cs *= f
        i = self.sigmoid(np.dot(self.i, self.x))
        c = self.tangent(np.dot(self.c, self.x))
        self.cs += i * c
        o = self.sigmoid(np.dot(self.o, self.x))
        self.y = o * self.tangent(self.cs)
        return self.cs, self.y, f, i, c, o
    
   
    def backProp(self, e, pcs, f, i, c, o, dfcs, dfhs):
        
        e = np.clip(e + dfhs, -6, 6)
        
        do = self.tangent(self.cs) * e
        
        ou = np.dot(np.atleast_2d(do * self.dtangent(o)).T, np.atleast_2d(self.x))
        
        dcs = np.clip(e * o * self.dtangent(self.cs) + dfcs, -6, 6)
        
        dc = dcs * i
    
        cu = np.dot(np.atleast_2d(dc * self.dtangent(c)).T, np.atleast_2d(self.x))
      
        di = dcs * c
        
        iu = np.dot(np.atleast_2d(di * self.dsigmoid(i)).T, np.atleast_2d(self.x))
       
        df = dcs * pcs
        
        fu = np.dot(np.atleast_2d(df * self.dsigmoid(f)).T, np.atleast_2d(self.x))
       
        dpcs = dcs * f
        
        dphs = np.dot(dc, self.c)[:self.output] + np.dot(do, self.o)[:self.output] + np.dot(di, self.i)[:self.output] + np.dot(df, self.f)[:self.output] 
       
        return fu, iu, cu, ou, dpcs, dphs
            
    def update(self, fu, iu, cu, ou):
        #Update forget, input, cell, and output gradients
        self.Gf = 0.9 * self.Gf + 0.1 * fu**2 
        self.Gi = 0.9 * self.Gi + 0.1 * iu**2   
        self.Gc = 0.9 * self.Gc + 0.1 * cu**2   
        self.Go = 0.9 * self.Go + 0.1 * ou**2   
        
        #Update our gates using our gradients
        self.f -= self.learning_rate/np.sqrt(self.Gf + 1e-8) * fu
        self.i -= self.learning_rate/np.sqrt(self.Gi + 1e-8) * iu
        self.c -= self.learning_rate/np.sqrt(self.Gc + 1e-8) * cu
        self.o -= self.learning_rate/np.sqrt(self.Go + 1e-8) * ou
        return


In [122]:
import numpy as np
RNN=RecurrentNeuralNetwork(5,5,1,np.array([6,6,6,6,6]),0.01)
RNN.setInput([1,2,3,4,5])
RNN.setTarget([6,6,6,6,6])
RNN.summary()
for i in range (100):
    RNN.forwardProp()
    print('_-----------------------------------------------------------------------------------------------------------------------------------------------------------------')

    RNN.backProp()
    RNN.summary()

--------------------------------------Initilizing LSTM---------------------
INPUT [1, 2, 3, 4, 5]
OUTPUT [6, 6, 6, 6, 6]
WEIGHT [[0.60707252 0.32621975 0.68102701 0.16850543 0.93177026]
 [0.93328422 0.64042781 0.43558787 0.60355025 0.323061  ]
 [0.69807727 0.99872546 0.79389041 0.35355377 0.0103463 ]
 [0.71979777 0.90956106 0.67247731 0.37255043 0.07318876]
 [0.48164365 0.14578063 0.06159735 0.85426597 0.32530801]]
G->matrix used in RMSprop in order to decay the learning rate [[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
ia->array for storing inputs [[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
Ca------>array for storing cell states [[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
oa--->array for storing outputs [[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
ha-->array for storing hidden state [[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
af-------> Forget gate [[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
ai----->input gate [[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
ac----->cell state [[0. 0. 0

In [115]:
RNN.LSTM.summary()

INPUT [0.94228499 0.99946948 0.99966475 0.99987325 0.99943218 0.
 0.         0.         0.         0.        ]
OUTPUT [0.75951492 0.75480006 0.75492405 0.76064863 0.76120137]
Initilizing cell state [0. 0. 0. 0. 0.]
f-------> Forget gate [[1.00210392 0.16933498 0.79819992 0.63063223 0.65917875 0.15954882
  0.19327806 0.00490554 0.37240952 0.14846618]
 [0.77503966 0.64617209 0.30134449 0.70458341 0.92711448 0.03453971
  0.89932759 0.20922965 0.76680155 0.40260719]
 [1.00208121 0.4557331  0.09162166 0.66446306 0.20406005 0.46127636
  0.5899049  0.33653729 0.92832671 0.32725737]
 [0.73898093 0.14360442 0.12810196 0.7180729  0.41541633 0.72368963
  0.48235167 0.21880624 0.89575598 0.60906589]
 [0.75962093 0.88095517 0.26741685 0.53174518 0.5619788  0.47168133
  0.61672736 0.11058501 0.9058614  0.36256012]]
i-------> input gate [[0.5303095  0.48917002 0.53625207 0.73868114 0.35901684 0.00818397
  0.2230614  0.05851438 0.80750761 0.85391392]
 [0.11902043 0.15471567 0.56984146 0.41719256 0.955

In [ ]:
a=np.array([0,1,1])
b=np.array([2,3,4])
a[0]

In [77]:
RNN.forwardProp()

AttributeError: 'RecurrentNeuralNetwork' object has no attribute 'LSTM'